In [1]:
import numpy as np
import pandas as pd

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('phishing.csv', index_col='Index')
df

,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,Favicon,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
Index,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,-1,0,-1,1,-1,-1,1,1,-1,1,...,1,1,1,1,1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11049,1,-1,1,-1,1,1,1,1,-1,-1,...,-1,-1,1,1,-1,-1,1,1,1,1
11050,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,-1
11051,1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,1,1,1,-1,1,0,1,-1


In [4]:
is_phishing = df[df['class'] == 1]
not_phishing = df[df['class'] == -1]

is_phishing_upsampled = resample(is_phishing, n_samples=len(not_phishing))

df_balanced = pd.concat([is_phishing_upsampled, not_phishing])
df_balanced

,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,Favicon,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
Index,,,,,,,,,,,,,,,,,,,,,
6027,1,-1,1,1,1,1,1,1,-1,1,...,1,1,-1,1,1,1,1,0,1,1
10385,-1,-1,1,1,1,-1,1,1,-1,-1,...,-1,1,1,1,1,-1,1,1,1,1
8564,-1,1,1,-1,1,1,1,1,-1,1,...,1,1,1,1,-1,1,1,1,1,1
3108,1,1,1,-1,1,-1,1,1,-1,1,...,1,1,1,1,1,1,1,0,1,1
4051,1,-1,1,-1,1,-1,-1,1,-1,1,...,1,1,1,1,1,-1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11048,-1,-1,1,1,-1,-1,1,-1,1,1,...,1,1,-1,1,1,-1,1,1,1,-1
11050,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,-1
11051,1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,1,1,1,-1,1,0,1,-1


In [6]:
params = ['UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//',
           'PrefixSuffix-', 'SubDomains', 'HTTPS', 'DomainRegLen', 'Favicon',
           'NonStdPort', 'HTTPSDomainURL', 'RequestURL', 'AnchorURL',
           'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL',
           'WebsiteForwarding', 'StatusBarCust', 'DisableRightClick',
           'UsingPopupWindow', 'IframeRedirection', 'AgeofDomain', 'DNSRecording',
           'WebsiteTraffic', 'PageRank', 'GoogleIndex', 'LinksPointingToPage',
           'StatsReport',]

x_data = df_balanced.drop(columns=['class'])
y_data = df_balanced['class']

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(x_train, y_train)

baseline_accuracy = accuracy_score(y_test, model.predict(x_test))
print("Baseline Accuracy:", baseline_accuracy)

for param in params:
    x_modified = x_data.drop(columns=[param])
    
    x_train_mod, x_test_mod, y_train_mod, y_test_mod = train_test_split(x_modified, y_data, test_size=0.2, random_state=42)
    
    model_mod = LogisticRegression()
    
    model_mod.fit(x_train_mod, y_train_mod)
    
    accuracy_mod = accuracy_score(y_test_mod, model_mod.predict(x_test_mod))
    
    if accuracy_mod < baseline_accuracy:
        print(f"Dropping {param} resulted in decreased accuracy: {accuracy_mod}")

Baseline Accuracy: 0.919346605410924
Dropping PrefixSuffix- resulted in decreased accuracy: 0.9116896375701888
Dropping HTTPS resulted in decreased accuracy: 0.8825931597753957
Dropping NonStdPort resulted in decreased accuracy: 0.9178152118427769
Dropping AnchorURL resulted in decreased accuracy: 0.8984175599795814
